# **Retrievers**

## What is Retrievers?

A retriever is a component in LangChain that fetches relevnt documents from a data source in reponse to a user's query. 


## **Wikipedia Retriever**

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
# Initialize the retriever (optional: set language and top_k)

retriever = WikipediaRetriever(top_k_results=3, lang='en')

In [3]:
# Define your query
query = "the geopolitical history of india and pakistan from the perspective of a chinese"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [4]:
docs

[Document(metadata={'title': 'Foreign relations of Pakistan', 'summary': "The Islamic Republic of Pakistan emerged as an independent country through the partition of India in August 1947 and was admitted as a United Nations member state in September 1947. It is currently the second-largest country within the Muslim world in terms of population, and is also the only Muslim-majority country in possession of nuclear weapons. De facto, the country shares direct land borders with India, Iran, Afghanistan, and China.\nThe country has extensive trade relations with the European Union and with several countries globally. As of 2023, Pakistan does not recognize two other United Nations member states (Armenia and Israel) and its ties with India remain frozen since 2019.\nFrom a geopolitical perspective, Pakistan's location is strategically important as it is situated at the crossroads of major maritime and land transit routes between the Middle East and South Asia, while also serving as a bridge

In [5]:
for i, doc in enumerate(docs):
    print(f"\n---Result {i + 1} ---")
    print(f"Content:\n{doc.page_content}...") ##Truncate for display


---Result 1 ---
Content:
The Islamic Republic of Pakistan emerged as an independent country through the partition of India in August 1947 and was admitted as a United Nations member state in September 1947. It is currently the second-largest country within the Muslim world in terms of population, and is also the only Muslim-majority country in possession of nuclear weapons. De facto, the country shares direct land borders with India, Iran, Afghanistan, and China.
The country has extensive trade relations with the European Union and with several countries globally. As of 2023, Pakistan does not recognize two other United Nations member states (Armenia and Israel) and its ties with India remain frozen since 2019.
From a geopolitical perspective, Pakistan's location is strategically important as it is situated at the crossroads of major maritime and land transit routes between the Middle East and South Asia, while also serving as a bridge between the Arabian Sea and the energy-rich regio

## **Vector Store Retriever**

In [6]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [8]:
embedding_model = OpenAIEmbeddings()

vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [9]:
retriever = vector_store.as_retriever(search_kwargs = {"k":2})

In [10]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [11]:
for i, doc in enumerate(results):
    print(f"\n---Result {i + 1} ---")
    print(f"Content:\n{doc.page_content}...") ##Truncate for display


---Result 1 ---
Content:
Chroma is a vector database optimized for LLM-based search....

---Result 2 ---
Content:
LangChain helps developers build LLM applications easily....


In [12]:
results = vector_store.similarity_search(query, k=2)

In [13]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


## **MMR (Maximal Marginal Relevance)**

In [14]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [15]:
from langchain_community.vectorstores import FAISS

embedding_model = OpenAIEmbeddings()

vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [16]:
retriever = vector_store.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 3, "lambda" : 0.5}
)

In [17]:
query = "What is langchain?"
results = retriever.invoke(query)

In [18]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
Embeddings are vector representations of text.

--- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


## **Multiquery Retriever**

In [20]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [21]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [22]:
embedding_model = OpenAIEmbeddings()

vector_store = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [23]:
similarity_retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k" : 5})

In [24]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs = {'k' : 5}),
    llm = ChatOpenAI(model="gpt-3.5-turbo")
)

In [25]:
# Query
query = "How to improve energy levels and maintain balance?"

In [26]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [27]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 4 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 5 ---
The solar energy system in modern homes helps balance electricity demand.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 3 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 4 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--

## **ContextualCompressionRetriever**

In [28]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [29]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [30]:
# Create a FAISS vector store from the documents
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

In [31]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [32]:
# Set up the compressor using an LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")
compressor = LLMChainExtractor.from_llm(llm)

In [33]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [34]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [35]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
